### What is Google Colab?

Google Colab is tool from Google Research that provides a jupyter-notebook style Python3 execution environment in your browser

### Creating a New Notebook

1. Go to https://colab.research.google.com/
2. Select "New notebook" at the bottom, OR select an existing notebook from the various sources (Google Drive / GitHub / Upload)

### Setting up the processor and run-time environment

1. Click on "Runtime" in the toolbar
2. Select "Change runtime type"
3. Select the appropriate option

### Interacting with the shell, installing **libraries**

In [ ]:
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! nvidia-smi 

Fri Aug  5 06:24:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Connecting to Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!rm -r /content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial

In [ ]:
import os
os.mkdir("/content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial")

In [ ]:
! ls /content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial

### Writing files to, and loading from, Google Drive

In [ ]:
import pandas as pd

In [ ]:
dummy_data = {"col_1":[1,2,3], "col_2":[10,20,4],}
df = pd.DataFrame(dummy_data)

In [ ]:
df.head()

col_1  col_2
0      1     10
1      2     20
2      3      4

In [ ]:
df.to_csv("/content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial/test.csv", index=False)

In [ ]:
! ls /content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial

test.csv


In [ ]:
df_load = pd.read_csv("/content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial/test.csv", index_col="col_1")
df_load.head()

col_2
col_1       
1         10
2         20
3          4

In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):

    def __init__(self, size):
        super(MLP, self).__init__()
        
        self.layers = []
        for in_dim, out_dim in zip(size[:-2], size[1:-1]):
          self.layers.extend([
              nn.Linear(in_dim, out_dim),
              nn.LeakyReLU(negative_slope=0.02),
              nn.BatchNorm1d(out_dim),
              nn.Dropout(0.5),
          ])
        self.layers.append(nn.Linear(size[-2], size[-1]))
        self.model = nn.Sequential(*self.layers)
        self.model.apply(self.init_param)

    def init_param(self, param):
      if type(param) == nn.Linear:
        nn.init.xavier_uniform_(param.weight)

    def forward(self, x):
      return self.model(x)

In [ ]:
model = MLP([40, 2048, 512, 256, 71])

In [ ]:
MODEL_SAVE_PATH = "/content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial/dummy_model.pt"

torch.save({
  'epoch': 10,
  'model_state_dict': model.state_dict(),
  'loss': 0.001,
}, MODEL_SAVE_PATH)

In [ ]:
! ls /content/drive/MyDrive/IDL-Fall-2022-Colab-Tutorial

dummy_model.pt	test.csv


In [ ]:
saved = torch.load(MODEL_SAVE_PATH)

print("epoch", saved["epoch"], ", loss", saved["loss"])


new_model = MLP([40, 2048, 512, 256, 71])
new_model.load_state_dict(saved["model_state_dict"])

epoch 10 , loss 0.001


<All keys matched successfully>

### Connecting to Kaggle

In [ ]:
import json

TOKEN = {"username":"adi1190","key":"0e8420354eb320bbdd2f609d5723ee21"}


! pip install kaggle==1.5.12
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=6d55667f4b8692311efa1b6e252a8b36e167403f197bbb3dbeca7a6250977000
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json
- path is now set to: /content


In [ ]:
! kaggle competitions download -c 11-785-fall-22-intro-to-colab

  0% 0.00/374 [00:00<?, ?B/s]
100% 374/374 [00:00<00:00, 426kB/s]


In [ ]:
! unzip competitions/11-785-fall-22-intro-to-colab/11-785-fall-22-intro-to-colab.zip

Archive:  competitions/11-785-fall-22-intro-to-colab/11-785-fall-22-intro-to-colab.zip
  inflating: sampleSubmission.csv    
  inflating: train.csv               


In [ ]:
! kaggle competitions submit -c 11-785-fall-22-intro-to-colab -f sampleSubmission.csv -m "A test submission"

100% 36.0/36.0 [00:03<00:00, 11.9B/s]
Successfully submitted to 11-785 Fall 22 Intro to colab

### Variable Inspector

In [ ]:
import numpy as np

In [ ]:
array = np.ones((10, 10))

### Resetting the runtime